In [ ]:
# The code was removed by Watson Studio for sharing.

##  Segmenting and Clustering Neighborhoods in Toronto Whose Postal Code Begins With the Letter M - Third Part

## The Following is from the First Part of Assignment

### 1.1 Creating Notebook with the Basic Dependencies

In [40]:
import numpy as np #library of handle data in a vectoized manner
import pandas as pd # library for data analysis
import requests # library for web scraping

print ('Libraries imported')

Libraries imported


### 1.2 Scraping Web Page Process Using BeautifulSoup

In [41]:
# installing Beautiful Soup for scraping data
# https://pypi.python.org/pypi/beautifulsoup4

# or download file
# http://beautiful-soup-4
# and unzip it in the same directory as this file

import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import csv
import json
import xml


print('BeautifulSoup and csv imported.')


BeautifulSoup and csv imported.


In [42]:
# Ignore SSL Certificate Errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

print('SSL Certificate Errors Ignored')

SSL Certificate Errors Ignored


In [43]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
#print(source)

soup = BeautifulSoup(source, 'lxml')
#print(soup)

#print(soup.prettify())
print('Soup Ready')

Soup Ready


In [44]:
table = soup.find('table',{'class':'wikitable sortable'})
#table
#print(table)

In [45]:
#from time import sleep
#sleep(3)

table_rows = table.find_all('tr')
#table rows

In [46]:
data=[]

for row in table_rows:
    data.append([t.text.strip() for t in row.find_all ('td')])
    
df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])    
df = df[~df['PostalCode'].isnull()] #to weed out bad rows

print(df.head(10))
print('****')
print(df.tail(10))

   PostalCode           Borough     Neighbourhood
1         M1A      Not assigned      Not assigned
2         M2A      Not assigned      Not assigned
3         M3A        North York         Parkwoods
4         M4A        North York  Victoria Village
5         M5A  Downtown Toronto      Harbourfront
6         M5A  Downtown Toronto       Regent Park
7         M6A        North York  Lawrence Heights
8         M6A        North York    Lawrence Manor
9         M7A      Queen's Park      Not assigned
10        M8A      Not assigned      Not assigned
****
    PostalCode       Borough             Neighbourhood
279        M4Z  Not assigned              Not assigned
280        M5Z  Not assigned              Not assigned
281        M6Z  Not assigned              Not assigned
282        M7Z  Not assigned              Not assigned
283        M8Z     Etobicoke  Kingsway Park South West
284        M8Z     Etobicoke                 Mimico NW
285        M8Z     Etobicoke        The Queensway West
286  

### 1.3.1 Process of Transforming Data into Pandas Dataframe

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 288 entries, 1 to 288
Data columns (total 3 columns):
PostalCode       288 non-null object
Borough          288 non-null object
Neighbourhood    288 non-null object
dtypes: object(3)
memory usage: 9.0+ KB


In [48]:
df.shape

(288, 3)

### 1.3.2 Dataframe With Unprocessed Data Established

In [49]:
import pandas
import requests
from bs4 import BeautifulSoup

website_text = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_text, 'lxml')

table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')

data=[]
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])
    
df = pandas.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood']) 
df = df[~df['PostalCode'].isnull()] #to filter out bad rows
df.head(15)



,PostalCode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned
10,M8A,Not assigned,Not assigned


### 1.3.3 Dropping Rows That Have Unassigned Buroughs

In [50]:
df.drop(df[df['Borough']=="Not assigned"].index, axis=0, inplace=True)

### 1.3.4 Reindex Dataframe

In [51]:
df1 = df.reset_index()
df1.head(10)

,index,PostalCode,Borough,Neighbourhood
0,3,M3A,North York,Parkwoods
1,4,M4A,North York,Victoria Village
2,5,M5A,Downtown Toronto,Harbourfront
3,6,M5A,Downtown Toronto,Regent Park
4,7,M6A,North York,Lawrence Heights
5,8,M6A,North York,Lawrence Manor
6,9,M7A,Queen's Park,Not assigned
7,11,M9A,Etobicoke,Islington Avenue
8,12,M1B,Scarborough,Rouge
9,13,M1B,Scarborough,Malvern


In [52]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211 entries, 0 to 210
Data columns (total 4 columns):
index            211 non-null int64
PostalCode       211 non-null object
Borough          211 non-null object
Neighbourhood    211 non-null object
dtypes: int64(1), object(3)
memory usage: 6.7+ KB


In [53]:
df1.shape

(211, 4)

### 1.3.5 Combine Multi Neighborhood Postal Codes

In [54]:
df2 = df1.groupby('PostalCode').agg(lambda x: ','.join(x))

df2.head(10)

,Borough,Neighbourhood
PostalCode,,
M1B,"Scarborough,Scarborough","Rouge,Malvern"
M1C,"Scarborough,Scarborough,Scarborough","Highland Creek,Rouge Hill,Port Union"
M1E,"Scarborough,Scarborough,Scarborough","Guildwood,Morningside,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,"Scarborough,Scarborough,Scarborough","East Birchmount Park,Ionview,Kennedy Park"
M1L,"Scarborough,Scarborough,Scarborough","Clairlea,Golden Mile,Oakridge"
M1M,"Scarborough,Scarborough,Scarborough","Cliffcrest,Cliffside,Scarborough Village West"


In [55]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 103 entries, M1B to M9W
Data columns (total 2 columns):
Borough          103 non-null object
Neighbourhood    103 non-null object
dtypes: object(2)
memory usage: 2.4+ KB


In [56]:
df2.shape


(103, 2)

### 1.3.6 For Boroughs That Have No Assigned Neighborhoods, Assign The Borough Name to the Neighborhood Section

In [57]:
df2.loc[df2['Neighbourhood']=='Not assigned','Neighbourhood']=df2.loc[df2['Neighbourhood']=="Not assigned",'Borough']
df2.head(10)

,Borough,Neighbourhood
PostalCode,,
M1B,"Scarborough,Scarborough","Rouge,Malvern"
M1C,"Scarborough,Scarborough,Scarborough","Highland Creek,Rouge Hill,Port Union"
M1E,"Scarborough,Scarborough,Scarborough","Guildwood,Morningside,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,"Scarborough,Scarborough,Scarborough","East Birchmount Park,Ionview,Kennedy Park"
M1L,"Scarborough,Scarborough,Scarborough","Clairlea,Golden Mile,Oakridge"
M1M,"Scarborough,Scarborough,Scarborough","Cliffcrest,Cliffside,Scarborough Village West"


In [58]:
df3 = df2.reset_index()
df3.head()

,PostalCode,Borough,Neighbourhood
0,M1B,"Scarborough,Scarborough","Rouge,Malvern"
1,M1C,"Scarborough,Scarborough,Scarborough","Highland Creek,Rouge Hill,Port Union"
2,M1E,"Scarborough,Scarborough,Scarborough","Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### 1.3.7 Remove Repeating Borough Names 

In [59]:
df3['Borough']=df3['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")

In [60]:
df3.head(10)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [61]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
PostalCode       103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
dtypes: object(3)
memory usage: 2.5+ KB


In [62]:
df3.shape

(103, 3)

### This Section Submitted to GetHub To Cover First Question of Assignment

## This Completes the First Part of Assignment

## The Following is the Second Part of Assignment

### 2.0 Getting CSV File That Contains Longitude and Latitude Values

In [63]:
!wget -q -O 'Toronto_long_lat_data.csv' http://cocl.us/Geospatial_data
df_lon_lat = pd.read_csv('Toronto_long_lat_data.csv')  
df_lon_lat.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [64]:
df_lon_lat.columns=['PostalCode', 'Latitude', 'Longitude']
df_lon_lat.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### 2.1 Merge Latitude and Longitude Information With Postal Code/Burough/Neighborhood Dataframe

In [65]:
Toronto_df = pd.merge(df3,
                     df_lon_lat[['PostalCode','Latitude','Longitude']],
                     on= 'PostalCode')
Toronto_df

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


### This Section Submitted to GetHub to Cover Second Question of Assignment

## The Completes Second Part of Assignment

## The Following is the Third Part of Assignment

### 3.0 Import Various Libraries

In [66]:
import numpy as np # library of handle data in a vectoized manor
import pandas as pd # library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json #library to handle JSON files
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim #convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize #transform JSON file into pandas dataframe

# matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans




Solving environment: done

# All requested packages already installed.



### 3.0.1 Determining One Address in Toronto

In [72]:
address = '162 North Queen Street, Etobicoke ON M9C 1A8, Canada'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('The geographical coordinate of 162 North Queen Street Etobicoke ON M9C 1A8, Canada is {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geographical coordinate of 162 North Queen Street Etobicoke ON M9C 1A8, Canada is 43.6200761, -79.5573552.


### 3.1 Install and Import Folium

In [73]:
!conda install -c conda-forge folium=0.5.0 
import folium #map rendering library
print('Libraries imported')

Solving environment: done

# All requested packages already installed.

Libraries imported


### 3.2 Visualizing Toronto "M" Points

In [74]:
# create map of Toronto using latitude and longitude values
Toronto_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighbourhood in zip(Toronto_df['Latitude'], Toronto_df['Longitude'], Toronto_df['Borough'], Toronto_df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Toronto_map)
Toronto_map 

### 3.3 Segmenting Etobicoke

In [75]:
Etobicoke_data = Toronto_df[Toronto_df.Borough == 'Etobicoke']
Etobicoke_data.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
88,M8V,Etobicoke,"Humber Bay Shores,Mimico South,New Toronto",43.605647,-79.501321
89,M8W,Etobicoke,"Alderwood,Long Branch",43.602414,-79.543484
90,M8X,Etobicoke,"The Kingsway,Montgomery Road,Old Mill North",43.653654,-79.506944
91,M8Y,Etobicoke,"Humber Bay,King's Mill Park,Kingsway Park Sout...",43.636258,-79.498509
92,M8Z,Etobicoke,"Kingsway Park South West,Mimico NW,The Queensw...",43.628841,-79.520999


In [76]:
Etobicoke=Etobicoke_data.reset_index()
Etobicoke = Etobicoke[Etobicoke.Neighbourhood != 'Islington Avenue']
Etobicoke

,index,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,88,M8V,Etobicoke,"Humber Bay Shores,Mimico South,New Toronto",43.605647,-79.501321
1,89,M8W,Etobicoke,"Alderwood,Long Branch",43.602414,-79.543484
2,90,M8X,Etobicoke,"The Kingsway,Montgomery Road,Old Mill North",43.653654,-79.506944
3,91,M8Y,Etobicoke,"Humber Bay,King's Mill Park,Kingsway Park Sout...",43.636258,-79.498509
4,92,M8Z,Etobicoke,"Kingsway Park South West,Mimico NW,The Queensw...",43.628841,-79.520999
6,94,M9B,Etobicoke,"Cloverdale,Islington,Martin Grove,Princess Gar...",43.650943,-79.554724
7,95,M9C,Etobicoke,"Bloordale Gardens,Eringate,Markland Wood,Old B...",43.643515,-79.577201
8,99,M9P,Etobicoke,Westmount,43.696319,-79.532242
9,100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.688905,-79.554724
10,101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437


In [77]:
Etobicoke.shape
print('The dataframe has', Etobicoke.shape, 'rows and columns respectively.')

The dataframe has (11, 6) rows and columns respectively.


### 3.4 Get the Latitude and Longitude Values of Etobicoke

In [78]:
address = 'Etobicoke, Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of Etobicoke are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geographical coordinate of Etobicoke are 43.67145915, -79.5524920661167.


### 3.5 Visualizating Etobicoke Sliced Data

In [79]:
map_Etobicoke = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Etobicoke['Latitude'], Etobicoke['Longitude'], Etobicoke['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Etobicoke)
map_Etobicoke

### 3.6 Etobicoke Neighborhood Cluster

### Foursquare API will be used to explore the neighborhoods and segment same

### 3.6.1 Define Foursquare Credentials and Version

In [ ]:
# Foursquare CLIENT_ID and CLIENT_SECRET information hidden

In [80]:
# The code was removed by Watson Studio for sharing.

### 3.6.2 Let's Explore the First Etobicoke Neighborhood in Our Dataframe

In [81]:
Etobicoke.loc[0,'Neighbourhood']

'Humber Bay Shores,Mimico South,New Toronto'

### 3.6.3 Get the First Neighbourhood's Latitude and Longitude Values

In [82]:
neighbourhood_latitude = Etobicoke.loc[0, 'Latitude'] # neighbourhood latitude value
neighbourhood_longitude = Etobicoke.loc[0, 'Longitude'] #neighbourhood longitude value

neighbourhood_name = Etobicoke.loc[0, 'Neighbourhood'] # neighbourhood name

print('The latitude and longitude values of {} are {},{}.'.format(neighbourhood_name,
                                                               neighbourhood_latitude,
                                                               neighbourhood_longitude))

The latitude and longitude values of Humber Bay Shores,Mimico South,New Toronto are 43.6056466,-79.50132070000001.


### 3.6.4 Let's Get the Top 100 Venues That are in the Humber Bay Shores/Mimico South/New Toronto Area Within a Radius of 500 Meters

### 3.6.4.1 Let's Create the GET Request URL, Named URL

In [83]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
#create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&Limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    neighbourhood_latitude,
    neighbourhood_longitude,
    radius,
    LIMIT)
# url #display URL

### 3.6.4.2 Send the GET Request and Examine the Results

In [84]:
results = requests.get(url).json()
#results

### 3.6.4.3 Before We Proceed, Let's Use the get_category_type Function to Get the Category Types

In [85]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### 3.6.4.4 Now We Can Clean the json and Structure it Into a Pandas Dataframe

In [86]:
venues = results['response']['groups'][0]['items']

nearby_venues = json_normalize(venues) # flatten JSON

#filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

#filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,LCBO,Liquor Store,43.602281,-79.499302
1,Domino's Pizza,Pizza Place,43.601676,-79.500908
2,New Toronto Fish & Chips,Restaurant,43.601849,-79.503281
3,Delicia Bakery & Pastry,Bakery,43.601403,-79.503012
4,Maple Leaf House,American Restaurant,43.602040,-79.498678


In [87]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

15 venues were returned by Foursquare.


### 3.7 Neighborhood Exploration

### 3.7.1 Let's Create a Function to Collect the Venues to All the Neighborhoods in Etobicoke

In [88]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        # create the API request url
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            LIMIT)
        
        # make the GET requst
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood',
                             'Neighbourhood Latitude',
                             'Neighbourhood Longitude',
                             'Venue',
                             'Venue Latitude',
                             'Venue Longitude',
                             'Venue Category']
                                 
    return(nearby_venues)    
            
        
                

In [89]:
Etobicoke

,index,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,88,M8V,Etobicoke,"Humber Bay Shores,Mimico South,New Toronto",43.605647,-79.501321
1,89,M8W,Etobicoke,"Alderwood,Long Branch",43.602414,-79.543484
2,90,M8X,Etobicoke,"The Kingsway,Montgomery Road,Old Mill North",43.653654,-79.506944
3,91,M8Y,Etobicoke,"Humber Bay,King's Mill Park,Kingsway Park Sout...",43.636258,-79.498509
4,92,M8Z,Etobicoke,"Kingsway Park South West,Mimico NW,The Queensw...",43.628841,-79.520999
6,94,M9B,Etobicoke,"Cloverdale,Islington,Martin Grove,Princess Gar...",43.650943,-79.554724
7,95,M9C,Etobicoke,"Bloordale Gardens,Eringate,Markland Wood,Old B...",43.643515,-79.577201
8,99,M9P,Etobicoke,Westmount,43.696319,-79.532242
9,100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.688905,-79.554724
10,101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437


### 3.7.2 Now Let's Write the Code to Run the Above Function on Each Neighborhood and Create a New Dataframe Called Etobicoke_venues

In [90]:
Etobicoke_venues = getNearbyVenues(names=Etobicoke['Neighbourhood'],
                                  latitudes=Etobicoke['Latitude'],
                                  longitudes=Etobicoke['Longitude']
                                  )

Humber Bay Shores,Mimico South,New Toronto
Alderwood,Long Branch
The Kingsway,Montgomery Road,Old Mill North
Humber Bay,King's Mill Park,Kingsway Park South East,Mimico NE,Old Mill South,The Queensway East,Royal York South East,Sunnylea
Kingsway Park South West,Mimico NW,The Queensway West,Royal York South West,South of Bloor
Cloverdale,Islington,Martin Grove,Princess Gardens,West Deane Park
Bloordale Gardens,Eringate,Markland Wood,Old Burnhamthorpe
Westmount
Kingsview Village,Martin Grove Gardens,Richview Gardens,St. Phillips
Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown
Northwest


In [91]:
print(Etobicoke_venues.shape)
Etobicoke_venues.head()

(75, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Humber Bay Shores,Mimico South,New Toronto",43.605647,-79.501321,LCBO,43.602281,-79.499302,Liquor Store
1,"Humber Bay Shores,Mimico South,New Toronto",43.605647,-79.501321,Domino's Pizza,43.601676,-79.500908,Pizza Place
2,"Humber Bay Shores,Mimico South,New Toronto",43.605647,-79.501321,New Toronto Fish & Chips,43.601849,-79.503281,Restaurant
3,"Humber Bay Shores,Mimico South,New Toronto",43.605647,-79.501321,Delicia Bakery & Pastry,43.601403,-79.503012,Bakery
4,"Humber Bay Shores,Mimico South,New Toronto",43.605647,-79.501321,Maple Leaf House,43.602040,-79.498678,American Restaurant


### 3.7.3 Let's Check How Many Venues Were Returned for Each Neighborhood

In [92]:
Etobicoke_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",9,9,9,9,9,9
"Alderwood,Long Branch",8,8,8,8,8,8
"Bloordale Gardens,Eringate,Markland Wood,Old Burnhamthorpe",9,9,9,9,9,9
"Cloverdale,Islington,Martin Grove,Princess Gardens,West Deane Park",1,1,1,1,1,1
"Humber Bay Shores,Mimico South,New Toronto",15,15,15,15,15,15
"Humber Bay,King's Mill Park,Kingsway Park South East,Mimico NE,Old Mill South,The Queensway East,Royal York South East,Sunnylea",4,4,4,4,4,4
"Kingsview Village,Martin Grove Gardens,Richview Gardens,St. Phillips",3,3,3,3,3,3
"Kingsway Park South West,Mimico NW,The Queensway West,Royal York South West,South of Bloor",14,14,14,14,14,14
Northwest,2,2,2,2,2,2


### 3.7.4 Let's Find Out How Many Unique Categories Can be Curated From all the Returned Venues

In [93]:
print('There are {} unique categories'.format(len(Etobicoke_venues['Venue Category'].unique())))

There are 40 unique categories


### 3.8 Lets Analyze Each Neighbourhood

In [94]:
# one hot encoding
Etobicoke_onehot = pd.get_dummies(Etobicoke_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Etobicoke_onehot['Neighbourhood'] = Etobicoke_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [Etobicoke_onehot.columns[-1]] + list(Etobicoke_onehot.columns[:-1])
Etobicoke_onehot = Etobicoke_onehot[fixed_columns]

Etobicoke_onehot.head()

,Neighbourhood,American Restaurant,Bakery,Bank,Baseball Field,Beer Store,Burger Joint,Burrito Place,Bus Line,Business Service,Café,Chinese Restaurant,Coffee Shop,Construction & Landscaping,Convenience Store,Discount Store,Drugstore,Fast Food Restaurant,Flower Shop,Fried Chicken Joint,Grocery Store,Gym,Hardware Store,Intersection,Liquor Store,Middle Eastern Restaurant,Park,Pet Store,Pharmacy,Pizza Place,Pool,Pub,Rental Car Location,Restaurant,River,Sandwich Place,Seafood Restaurant,Skating Rink,Supplement Shop,Tanning Salon,Wings Joint
0,"Humber Bay Shores,Mimico South,New Toronto",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Humber Bay Shores,Mimico South,New Toronto",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,"Humber Bay Shores,Mimico South,New Toronto",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,"Humber Bay Shores,Mimico South,New Toronto",0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Humber Bay Shores,Mimico South,New Toronto",1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### 3.8.1 Lets Examine the New Dataframe Size

In [95]:
Etobicoke_onehot.shape
print('The dataframe has', Etobicoke_onehot.shape, 'rows and columns respectively.')

The dataframe has (75, 41) rows and columns respectively.


### 3.8.2 Next, Let's Group Rows by Neighborhood by Taking the Mean of the Frequency of Occurrence of Each Category

In [96]:
Etobicoke_grouped = Etobicoke_onehot.groupby('Neighbourhood').mean().reset_index()
Etobicoke_grouped

,Neighbourhood,American Restaurant,Bakery,Bank,Baseball Field,Beer Store,Burger Joint,Burrito Place,Bus Line,Business Service,Café,Chinese Restaurant,Coffee Shop,Construction & Landscaping,Convenience Store,Discount Store,Drugstore,Fast Food Restaurant,Flower Shop,Fried Chicken Joint,Grocery Store,Gym,Hardware Store,Intersection,Liquor Store,Middle Eastern Restaurant,Park,Pet Store,Pharmacy,Pizza Place,Pool,Pub,Rental Car Location,Restaurant,River,Sandwich Place,Seafood Restaurant,Skating Rink,Supplement Shop,Tanning Salon,Wings Joint
0,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.000000,0.000000,0.0,0.00,0.111111,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.111111,0.00,0.000000,0.000000,0.0,0.111111,0.000000,0.111111,0.222222,0.000000,0.000000,0.000,0.000000,0.000,0.000000,0.000000,0.111111,0.111111,0.00,0.000,0.0,0.000000,0.0,0.111111,0.000000,0.000,0.000000,0.000000,0.000000
1,"Alderwood,Long Branch",0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.125000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.000,0.000000,0.000,0.000000,0.000000,0.125000,0.250000,0.00,0.125,0.0,0.000000,0.0,0.125000,0.000000,0.125,0.000000,0.000000,0.000000
2,"Bloordale Gardens,Eringate,Markland Wood,Old B...",0.000000,0.000000,0.0,0.00,0.111111,0.000000,0.000000,0.000000,0.00,0.111111,0.000,0.111111,0.00,0.111111,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.111111,0.000,0.111111,0.111111,0.111111,0.111111,0.00,0.000,0.0,0.000000,0.0,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
3,"Cloverdale,Islington,Martin Grove,Princess Gar...",0.000000,0.000000,1.0,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.0,0.000000,0.0,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
4,"Humber Bay Shores,Mimico South,New Toronto",0.066667,0.066667,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.133333,0.000,0.066667,0.00,0.000000,0.000000,0.0,0.066667,0.000000,0.066667,0.000000,0.066667,0.000000,0.000,0.066667,0.000,0.000000,0.066667,0.066667,0.066667,0.00,0.000,0.0,0.066667,0.0,0.066667,0.066667,0.000,0.000000,0.000000,0.000000
5,"Humber Bay,King's Mill Park,Kingsway Park Sout...",0.000000,0.000000,0.0,0.25,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000,0.000000,0.25,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.25,0.000,0.0,0.000000,0.0,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
6,"Kingsview Village,Martin Grove Gardens,Richvie...",0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.333333,0.00,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000,0.333333,0.000000,0.000000,0.333333,0.00,0.000,0.0,0.000000,0.0,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
7,"Kingsway Park South West,Mimico NW,The Queensw...",0.000000,0.071429,0.0,0.00,0.000000,0.071429,0.071429,0.000000,0.00,0.000000,0.000,0.000000,0.00,0.071429,0.071429,0.0,0.071429,0.071429,0.000000,0.071429,0.071429,0.071429,0.000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.0,0.000000,0.0,0.071429,0.000000,0.000,0.071429,0.071429,0.071429
8,Northwest,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.5,0.000000,0.0,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
9,"The Kingsway,Montgomery Road,Old Mill North",0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000,0.500000,0.000000,0.000000,

### 3.8.3 Let's Print Each Neighborhood Along With the Top 5 Most Common Venues

In [97]:
num_top_venues = 5

for hood in Etobicoke_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = Etobicoke_grouped[Etobicoke_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                 venue  freq
0        Grocery Store  0.22
1          Coffee Shop  0.11
2           Beer Store  0.11
3  Fried Chicken Joint  0.11
4       Sandwich Place  0.11


----Alderwood,Long Branch----
            venue  freq
0     Pizza Place  0.25
1             Gym  0.12
2  Sandwich Place  0.12
3        Pharmacy  0.12
4     Coffee Shop  0.12


----Bloordale Gardens,Eringate,Markland Wood,Old Burnhamthorpe----
         venue  freq
0  Coffee Shop  0.11
1         Café  0.11
2         Park  0.11
3    Pet Store  0.11
4     Pharmacy  0.11


----Cloverdale,Islington,Martin Grove,Princess Gardens,West Deane Park----
                       venue  freq
0                       Bank   1.0
1        American Restaurant   0.0
2                        Pub   0.0
3               Liquor Store   0.0
4  Middle Eastern Restaurant   0.0


----Humber Bay Shores,Mimico South,New Toronto----
    

### 3.8.4 Let's Put That Into a Pandas Dataframe

### 3.8.4.1 First, Let's Write a Function to Sort the Venues in Descending Order

In [98]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### 3.8.4.2 Now Let's Create the New Dataframe and Display the Top 10 Venues for Each Neighborhood

In [99]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = Etobicoke_grouped['Neighbourhood']

for ind in np.arange(Etobicoke_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Etobicoke_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Beer Store,Fast Food Restaurant,Coffee Shop,Pharmacy,Pizza Place,Sandwich Place,Fried Chicken Joint,Burger Joint,Burrito Place
1,"Alderwood,Long Branch",Pizza Place,Skating Rink,Sandwich Place,Gym,Coffee Shop,Pub,Pharmacy,Business Service,Discount Store,Convenience Store
2,"Bloordale Gardens,Eringate,Markland Wood,Old B...",Pizza Place,Pharmacy,Convenience Store,Beer Store,Coffee Shop,Liquor Store,Park,Café,Pet Store,Discount Store
3,"Cloverdale,Islington,Martin Grove,Princess Gar...",Bank,Wings Joint,Chinese Restaurant,Flower Shop,Fast Food Restaurant,Drugstore,Discount Store,Convenience Store,Construction & Landscaping,Coffee Shop
4,"Humber Bay Shores,Mimico South,New Toronto",Café,American Restaurant,Pizza Place,Fast Food Restaurant,Fried Chicken Joint,Coffee Shop,Gym,Liquor Store,Pet Store,Pharmacy
5,"Humber Bay,King's Mill Park,Kingsway Park Sout...",Pool,Baseball Field,Construction & Landscaping,Business Service,Café,Fast Food Restaurant,Drugstore,Discount Store,Convenience Store,Coffee Shop
6,"Kingsview Village,Martin Grove Gardens,Richvie...",Pizza Place,Park,Bus Line,Wings Joint,Fast Food Restaurant,Drugstore,Discount Store,Convenience Store,Construction & Landscaping,Coffee Shop
7,"Kingsway Park South West,Mimico NW,The Queensw...",Wings Joint,Hardware Store,Bakery,Burger Joint,Burrito Place,Convenience Store,Discount Store,Fast Food Restaurant,Flower Shop,Tanning Salon
8,Northwest,Drugstore,Rental Car Location,Wings Joint,Café,Fast Food Restaurant,Discount Store,Convenience Store,Construction & Landscaping,Coffee Shop,Chinese Restaurant
9,"The Kingsway,Montgomery Road,Old Mill North",River,Park,Wings Joint,Café,Fast Food Restaurant,Drugstore,Discount Store,Convenience Store,Construction & Landscaping,Coffee Shop


In [100]:
neighbourhoods_venues_sorted.shape
print('The dataframe has', neighbourhoods_venues_sorted.shape, 'rows and columns respectively.')

The dataframe has (11, 11) rows and columns respectively.


### 3.9 Cluster Neighborhoods

### 3.9.1 Run k-means to Cluster the Neighborhood Into 5 Clusters

In [101]:

# set number of clusters
kclusters = 5

Etobicoke_grouped_clustering = Etobicoke_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Etobicoke_grouped_clustering)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_[0:10]
kmeans.labels_[0:15]

array([3, 3, 3, 2, 3, 0, 3, 3, 4, 1, 3], dtype=int32)

In [123]:
Etobicoke

,index,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,88,M8V,Etobicoke,"Humber Bay Shores,Mimico South,New Toronto",43.605647,-79.501321
1,89,M8W,Etobicoke,"Alderwood,Long Branch",43.602414,-79.543484
2,90,M8X,Etobicoke,"The Kingsway,Montgomery Road,Old Mill North",43.653654,-79.506944
3,91,M8Y,Etobicoke,"Humber Bay,King's Mill Park,Kingsway Park Sout...",43.636258,-79.498509
4,92,M8Z,Etobicoke,"Kingsway Park South West,Mimico NW,The Queensw...",43.628841,-79.520999
6,94,M9B,Etobicoke,"Cloverdale,Islington,Martin Grove,Princess Gar...",43.650943,-79.554724
7,95,M9C,Etobicoke,"Bloordale Gardens,Eringate,Markland Wood,Old B...",43.643515,-79.577201
8,99,M9P,Etobicoke,Westmount,43.696319,-79.532242
9,100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.688905,-79.554724
10,101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437


### 3.9.2 Let's Create a New Dataframe That Includes the Cluster as Well as the Top 10 Venues for Each Neighborhood

In [102]:
Etobicoke_merged = Etobicoke

# add clustering labels
Etobicoke_merged['Cluster Labels'] = kmeans.labels_

# merge Etobicoke_grouped with Etobicoke_data to add latitude/longitude for each neighborhood
Etobicoke_merged = Etobicoke_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

Etobicoke_merged.head() # check the last columns!

,index,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,88,M8V,Etobicoke,"Humber Bay Shores,Mimico South,New Toronto",43.605647,-79.501321,3,Café,American Restaurant,Pizza Place,Fast Food Restaurant,Fried Chicken Joint,Coffee Shop,Gym,Liquor Store,Pet Store,Pharmacy
1,89,M8W,Etobicoke,"Alderwood,Long Branch",43.602414,-79.543484,3,Pizza Place,Skating Rink,Sandwich Place,Gym,Coffee Shop,Pub,Pharmacy,Business Service,Discount Store,Convenience Store
2,90,M8X,Etobicoke,"The Kingsway,Montgomery Road,Old Mill North",43.653654,-79.506944,3,River,Park,Wings Joint,Café,Fast Food Restaurant,Drugstore,Discount Store,Convenience Store,Construction & Landscaping,Coffee Shop
3,91,M8Y,Etobicoke,"Humber Bay,King's Mill Park,Kingsway Park Sout...",43.636258,-79.498509,2,Pool,Baseball Field,Construction & Landscaping,Business Service,Café,Fast Food Restaurant,Drugstore,Discount Store,Convenience Store,Coffee Shop
4,92,M8Z,Etobicoke,"Kingsway Park South West,Mimico NW,The Queensw...",43.628841,-79.520999,3,Wings Joint,Hardware Store,Bakery,Burger Joint,Burrito Place,Convenience Store,Discount Store,Fast Food Restaurant,Flower Shop,Tanning Salon


### 3.9.3 Finally, Let's Visualize the Resulting Clusters

In [103]:
# create map
map_Etobicoke_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Etobicoke_merged['Latitude'], Etobicoke_merged['Longitude'], Etobicoke_merged['Neighbourhood'], Etobicoke_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_Etobicoke_clusters)
       
map_Etobicoke_clusters

### 3.10 Let's Examine Clusters


### Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster. 

### 3.10.1 Cluster 1

In [104]:
Etobicoke_merged.loc[Etobicoke_merged['Cluster Labels'] == 0, Etobicoke_merged.columns[[1] + list(range(5, Etobicoke_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,M9B,-79.554724,0,Bank,Wings Joint,Chinese Restaurant,Flower Shop,Fast Food Restaurant,Drugstore,Discount Store,Convenience Store,Construction & Landscaping,Coffee Shop


### 3.10.2 Cluster 2

In [105]:
Etobicoke_merged.loc[Etobicoke_merged['Cluster Labels'] == 1, Etobicoke_merged.columns[[1] + list(range(5, Etobicoke_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,M9V,-79.588437,1,Grocery Store,Beer Store,Fast Food Restaurant,Coffee Shop,Pharmacy,Pizza Place,Sandwich Place,Fried Chicken Joint,Burger Joint,Burrito Place


### 3.10.3 Cluster 3

In [106]:
Etobicoke_merged.loc[Etobicoke_merged['Cluster Labels'] == 2, Etobicoke_merged.columns[[1] + list(range(5, Etobicoke_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,M8Y,-79.498509,2,Pool,Baseball Field,Construction & Landscaping,Business Service,Café,Fast Food Restaurant,Drugstore,Discount Store,Convenience Store,Coffee Shop


### 3.10.4 Cluster 4

In [107]:
Etobicoke_merged.loc[Etobicoke_merged['Cluster Labels'] == 3, Etobicoke_merged.columns[[1] + list(range(5, Etobicoke_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M8V,-79.501321,3,Café,American Restaurant,Pizza Place,Fast Food Restaurant,Fried Chicken Joint,Coffee Shop,Gym,Liquor Store,Pet Store,Pharmacy
1,M8W,-79.543484,3,Pizza Place,Skating Rink,Sandwich Place,Gym,Coffee Shop,Pub,Pharmacy,Business Service,Discount Store,Convenience Store
2,M8X,-79.506944,3,River,Park,Wings Joint,Café,Fast Food Restaurant,Drugstore,Discount Store,Convenience Store,Construction & Landscaping,Coffee Shop
4,M8Z,-79.520999,3,Wings Joint,Hardware Store,Bakery,Burger Joint,Burrito Place,Convenience Store,Discount Store,Fast Food Restaurant,Flower Shop,Tanning Salon
7,M9C,-79.577201,3,Pizza Place,Pharmacy,Convenience Store,Beer Store,Coffee Shop,Liquor Store,Park,Café,Pet Store,Discount Store
8,M9P,-79.532242,3,Pizza Place,Discount Store,Sandwich Place,Intersection,Middle Eastern Restaurant,Coffee Shop,Chinese Restaurant,Café,Drugstore,Convenience Store
11,M9W,-79.594054,3,Drugstore,Rental Car Location,Wings Joint,Café,Fast Food Restaurant,Discount Store,Convenience Store,Construction & Landscaping,Coffee Shop,Chinese Restaurant


### 3.10.5 Cluster 5

In [108]:
Etobicoke_merged.loc[Etobicoke_merged['Cluster Labels'] == 4, Etobicoke_merged.columns[[1] + list(range(5, Etobicoke_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,M9R,-79.554724,4,Pizza Place,Park,Bus Line,Wings Joint,Fast Food Restaurant,Drugstore,Discount Store,Convenience Store,Construction & Landscaping,Coffee Shop
